In [0]:
import pandas as pd
import random
from datetime import datetime, timedelta


storageAccountName = "azemourstorage"
storageAccountAccessKey = "kEjUrj5seRnPiSRkcQRPmozB610QhZLtZn9GrRhZk/D76nVsr6DNejvGRq0wEQWFiqcJjWgWi7cQ+AStydUbmQ=="
sasToken = "?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2023-09-27T17:28:21Z&st=2023-09-27T09:28:21Z&spr=https&sig=EfKk2ZhRwqQFy228Dt7Z19oDuI61hNTNiQkAinm2nBc%3D"
blobContainerName = "public-transport-data"
mountPoint = "/mnt/public-transport-data/"
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
      mount_point = mountPoint,
      extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)


# Generate data for January 2023
start_date = datetime(2023, 2, 1)
end_date = datetime(2023, 2, 30)
date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date - start_date).days)]

transport_types = ["Bus", "Train", "Tram", "Metro"]
routes = ["Route_" + str(i) for i in range(1, 11)]
stations = ["Station_" + str(i) for i in range(1, 21)]

# Randomly select 5 days as extreme weather days
extreme_weather_days = random.sample(date_generated, 5)

data = []

for date in date_generated:
    for _ in range(32):  # 32 records per day to get a total of 992 records for January
        transport = random.choice(transport_types)
        route = random.choice(routes)

        # Normal duration
        duration = random.randint(10, 120)

        # Introducing Short Turnarounds
        if random.random() < 0.05:  # 5% chance
            duration = random.randint(1, 5)

        # General delay
        delay = random.randint(0, 15)

        # Calculate arrival hour and minute
        total_minutes = duration + delay  # Moved calculation after defining duration and delay
        departure_hour = random.randint(5, 22)
        departure_minute = random.randint(0, 59)
        arrival_hour = departure_hour + total_minutes // 60
        arrival_minute = total_minutes % 60

        # Adjust arrival hour and minute if needed
        if arrival_hour >= 24:
            arrival_hour -= 24

        # Format departure and arrival times
        departure_time = datetime.strptime(f"{departure_hour:02}:{departure_minute:02}", "%H:%M").strftime("%I:%M:%S %p")
        arrival_time = datetime.strptime(f"{arrival_hour:02}:{arrival_minute:02}", "%H:%M").strftime("%I:%M:%S %p")

        # Weather Impact
        if date in extreme_weather_days:
            # Increase delay by 10 to 60 minutes
            delay += random.randint(10, 60)

            # 10% chance to change the route
            if random.random() < 0.10:
                route = random.choice(routes)

        passengers = random.randint(1, 100)
        departure_station = random.choice(stations)
        arrival_station = random.choice(stations)

        data.append([date, transport, route, departure_time, arrival_time, passengers, departure_station, arrival_station, delay])

# Generate the output file name with the month number
month_number = start_date.month
file_path = f"/dbfs/mnt/public-transport-data/raw/public_transport_data_month_{month_number}.csv"

# Création d'un DataFrame Spark à partir des données
df = spark.createDataFrame(data, schema=["Date", "TransportType", "Route", "DepartureTime", "ArrivalTime", "Passengers", "DepartureStation", "ArrivalStation", "Delay"])

data = df.toPandas()
data.to_csv(file_path, index=False)
print("Terminé")